In [1]:
##Library
import pandas as pd
import numpy as np
import cv2
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from facenet_pytorch import MTCNN
import matplotlib.pyplot as plt

/Users/sishengliang/opt/anaconda3/envs/happiness-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
##Data Prepare
# Load FER+ dataset
df = pd.read_csv("data/fer2013.csv")

In [ ]:
# Emotion to happiness score mapping
emotion_map = {
    "happy": 9,
    "neutral": 6,
    "surprise": 7,
    "angry": 2,
    "sad": 2,
    "fear": 3,
    "disgust": 1,
}

In [ ]:
# Convert categorical emotions to numerical scores
df['happiness_score'] = df['emotion'].map(emotion_map)

# Convert pixel strings to numpy arrays
def process_pixels(pixel_string):
    pixels = np.array(pixel_string.split(), dtype=np.uint8).reshape(48, 48)
    return cv2.resize(pixels, (224, 224))  # Resize for deep learning models

df['pixels'] = df['pixels'].apply(process_pixels)


In [ ]:
# Save cleaned data
df.to_pickle("data/fer_cleaned.pkl")

#### Why? •	Converts FER+ emotions into happiness scores.
####	  •	Resizes images to 224x224 for CNN models.

In [ ]:
class HappinessDataset(Dataset):
    def __init__(self, pickle_file, transform=None):
        self.data = pd.read_pickle(pickle_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_array = self.data.iloc[idx]["pixels"]
        img = cv2.cvtColor(img_array, cv2.COLOR_GRAY2RGB)  # Convert to 3-channel

        if self.transform:
            img = self.transform(img)

        score = torch.tensor(self.data.iloc[idx]["happiness_score"], dtype=torch.float)
        return img, score

# Define transforms for data augmentation
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Load dataset
dataset = HappinessDataset("data/fer_cleaned.pkl", transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Test data loading
sample_img, sample_score = dataset[0]
plt.imshow(sample_img.permute(1, 2, 0))  # Convert tensor to image
plt.title(f"Happiness Score: {sample_score.item()}")
plt.show()

In [ ]:
###Training the Happiness Score Model
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

# Define model (ResNet50 pretrained)
class HappinessModel(nn.Module):
    def __init__(self):
        super(HappinessModel, self).__init__()
        self.model = models.resnet50(pretrained=True)
        self.model.fc = nn.Linear(2048, 1)  # Regression output

    def forward(self, x):
        return self.model(x)

# Training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HappinessModel().to(device)

criterion = nn.MSELoss()  # Regression loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10):
    for images, scores in dataloader:
        images, scores = images.to(device), scores.to(device)

        optimizer.zero_grad()
        outputs = model(images).squeeze()
        loss = criterion(outputs, scores)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")
    
# Save model
torch.save(model.state_dict(), "happiness_model.pth")

###	•	Uses ResNet50 for feature extraction.
### •	Predicts continuous happiness scores (1-10).
### •	Uses MSELoss for regression.